# Peptidesim simulation

## We will run a simulation of FF peptide with amber99sb forcefield, with tip4p water model

### 0) For this tutorial you need a gromacs, plumed, and peptidesim installed

Here is a sample bash script that needs to be run before starting the jupyter notebook

```
#!/bin/bash
module unload python
module load anaconda3 packmol libmatheval gromacs-plumed/2019.4/b2 openblas git
export PYTHONNOUSERSITE=True
export OMP_NUM_THREADS=1
source activate your_environment 
pip install mdanalysis
pip install nglview
pip install moviepy==0.2.2.11
pip install imageio==1.6
```

In [1]:
import peptidesim
import textwrap
from peptidesim import PeptideSim

In [2]:
mkdir ~/tutorials

mkdir: cannot create directory ‘/home/damirkul/tutorials’: File exists


In [3]:
cd tutorials

/gpfs/fs1/home/damirkul/ab42_fragments/tutorials


### 1) Defining the peptidesim object
* Let's create a simulation called peptide_tutorial

In [4]:
ps = PeptideSim('peptide_tutorial', ['FF'], [1],job_name='peptide_tutorial')


### 2) Changing the attributes of the object
* change the forcefield from default (charmm27) to amber99sb
* change the water model from tip3p to tip4p
* change the density and concentration
* we take advantage of parallel processes hence we will call `gmx_mpi`

In [5]:
print(ps)

==========PeptideSim Object==========
box_size_nm    : [2.49963434652162, 2.499634346521621, 2.499634346521621]
config         : {}
config_file    : peptidesim_config.py
counts         : [1]
current_gro    : peptide_tutorial/peptidesim_nvt-378b483c/peptidesim_nvt.gro
current_pdb    : peptide_tutorial/peptide_structures/dry_packed.pdb
current_top    : peptide_tutorial/prep/pp_dry_topology.top
current_tpr    : peptide_tutorial/prep/ion.tpr
current_traj   : peptide_tutorial/peptidesim_nvt-378b483c/traj.trr
demux_exe      : demux
dir_name       : peptide_tutorial
file_list      : ['peptide_tutorial/prep/index.ndx', 'traj.trr', 'traj.trr', 'traj.trr', 'traj.trr', 'plumed.dat', 'traj.trr', 'peptide_tutorial/prep/index.ndx', 'plumed.dat']
forcefield     : charmm27
ion_concentration : 0.002
job_name       : peptide_tutorial
log_file       : simulation.log
mdp_base       : peptidesim_base.mdp
mdp_directory  : .
mdp_emin       : peptidesim_emin.mdp
mdrun_driver   : None
mpi_np         : 1
mpiexe

In [6]:
ps.forcefield = 'amber99sb'
ps.water = 'tip4p'
ps.ion_concentration = 0.001  # 10mM
ps.mpi_np=1
ps.run_kwargs={'nt':1}

### 3) Initialize the object

In [7]:
ps.initialize()

In [8]:
ls peptide_tutorial/

anneal_nvt-378b483c/       peptidesim_nvt-378b483c/         prep/
init_emin-378b483c/        peptide_structures/              restarts/
initialize-emin-378b483c/  peptide_tutorial-restart.pickle  simulation.log
nvt_prod-378b483c/         peptide_tutorial-status.txt


In [9]:
pwd

'/gpfs/fs1/home/damirkul/ab42_fragments/tutorials'

### 4) We will run classic set of Molecular Dynamics simulation 
*  Energy Minimizization

`emtol = 1e-10
nsteps = 100
nstxout = 1
nstenergy = 1
nstlog = 0
integrator = steep
constraints = none`
*  Annealing

`integrator   = sd
nstxout      = 25
nstvout      = 0
nstfout      = 0
nstlog       = 25
nstenergy    = 0
tcoupl       = v-rescale
tau-t        = 2
tc-grps      = System
ref-p        = 0
constraints  = h-bonds
dt           = 0.002
annealing    = single
ref-t        = 300
annealing-npoints = 3
nsteps       = 2500
annealing-time = 0 2.5 5
annealing-temp = 350 450 300
lincs-iter     = 3 `
*  NVT Production at 400K

`  
integrator   = sd
nsteps       = 1
dt           = 0.002
nstxout      = 1000
nstvout      = 0
nstfout      = 0
nstlog       = 1000
nstenergy    = 0
tcoupl       = v-rescale
tau_t        = 2
ref_t        = 300
tc-grps      = System
ref_p        = 0`

In [10]:
ps.run(mdpfile='peptidesim_emin.mdp', tag='init_emin', mdp_kwargs={'nsteps': 10**2})
ps.run(mdpfile='peptidesim_anneal.mdp', tag='anneal_nvt',mdp_kwargs={'nsteps': 10**2})
ps.run(mdpfile='peptidesim_nvt.mdp', tag='nvt_prod', mdp_kwargs={'nsteps': int(3 * 5*10**2),'ref_t':400})

In [11]:
ls -lrt

total 24336
-rw-------   1 damirkul damirkul 24891900 Mar  9 13:38 traj.trr
drwx------+  6 damirkul damirkul      512 Mar  9 15:00 peptide_tutorial_2/
drwx------+ 10 damirkul damirkul    16384 Mar  9 15:19 peptide_tutorial/
-rw-------   1 damirkul damirkul       64 Mar  9 16:23 plumed.dat


In [12]:
ls -lrt

total 24336
-rw-------   1 damirkul damirkul 24891900 Mar  9 13:38 traj.trr
drwx------+  6 damirkul damirkul      512 Mar  9 15:00 peptide_tutorial_2/
drwx------+ 10 damirkul damirkul    16384 Mar  9 15:19 peptide_tutorial/
-rw-------   1 damirkul damirkul       64 Mar  9 16:23 plumed.dat


## Printing the Peptide object

In [13]:
print(ps)

==========PeptideSim Object==========
box_size_nm    : [2.49963434652162, 2.499634346521621, 2.499634346521621]
config         : {}
config_file    : peptidesim_config.py
counts         : [1]
current_gro    : peptide_tutorial/nvt_prod-378b483c/nvt_prod.gro
current_pdb    : peptide_tutorial/peptide_structures/dry_packed.pdb
current_top    : peptide_tutorial/prep/pp_dry_topology.top
current_tpr    : peptide_tutorial/prep/ion.tpr
current_traj   : peptide_tutorial/peptidesim_nvt-378b483c/traj.trr
demux_exe      : demux
dir_name       : peptide_tutorial
file_list      : ['peptide_tutorial/prep/index.ndx', 'traj.trr', 'traj.trr', 'traj.trr', 'traj.trr', 'plumed.dat', 'traj.trr', 'peptide_tutorial/prep/index.ndx', 'plumed.dat', 'peptide_tutorial/prep/index.ndx']
forcefield     : charmm27
ion_concentration : 0.002
job_name       : peptide_tutorial
log_file       : simulation.log
mdp_base       : peptidesim_base.mdp
mdp_directory  : .
mdp_emin       : peptidesim_emin.mdp
mdrun_driver   : None
mp

### 5) How to add a plumed script to a simulation
* Let's compute a distance between 1st and 2nd atoms during the simulation

In [14]:
plumed_file='plumed.dat' # name of the plumed file
## computing the distance between atoms
plumed_input = textwrap.dedent(
    '''
    DISTANCE ATOMS=1,2 LABEL=d1
    PRINT ARG=d1 FILE=colvar STRIDE=10
    ''')

#we need to write the above string to a file
with open(plumed_file, 'w') as f:
    f.write(plumed_input)
# we need to add the plumed file to a list of required files
ps.add_file(plumed_file)
ps.run(
    mdpfile='peptidesim_nvt.mdp',
    mdp_kwargs={'nsteps':1000,'nstxout':100, 'nstvout':100, 'nstfout':100, 'nstlog':100},
    run_kwargs={
        'plumed': plumed_file})

In [15]:
pwd


'/gpfs/fs1/home/damirkul/ab42_fragments/tutorials'

In [16]:
print(ps.sims[-1].location)

peptide_tutorial/peptidesim_nvt-378b483c


In [17]:
### !!!change the diretory to the latest simulation folder (the output of `ps.sims[-1].location)



In [18]:
cd peptide_tutorial/peptidesim_nvt-378b483c

/gpfs/fs1/home/damirkul/ab42_fragments/tutorials/peptide_tutorial/peptidesim_nvt-378b483c


In [19]:
ls -lrt

total 26704
-rw-------   1 damirkul damirkul 24891900 Mar  9 13:38 #traj.trr.1#
-rw-------   1 damirkul damirkul    41939 Mar  9 15:11 index.ndx
-rw-------   1 damirkul damirkul    65868 Mar  9 15:19 #peptidesim_nvt.tpr.1#
-rw-------   1 damirkul damirkul     1937 Mar  9 15:20 bck.0.colvar
-rw-------   1 damirkul damirkul   811932 Mar  9 15:20 #traj.trr.2#
-rw-------   1 damirkul damirkul   141308 Mar  9 15:20 #peptidesim_nvt.gro.1#
-rw-------   1 damirkul damirkul     1412 Mar  9 15:20 #ener.edr.1#
-rw-------   1 damirkul damirkul    29901 Mar  9 15:20 #md.log.1#
-rw-------   1 damirkul damirkul      232 Mar  9 16:17 peptidesim_base.mdp
-rw-------   1 damirkul damirkul      370 Mar  9 16:17 peptidesim_nvt.mdp
-rw-------   1 damirkul damirkul    10990 Mar  9 16:17 mdout.mdp
-rw-------   1 damirkul damirkul    65868 Mar  9 16:17 peptidesim_nvt.tpr
-rw-------   1 damirkul damirkul     1937 Mar  9 16:17 colvar
-rw-------   1 damirkul damirkul    50640 Mar  9 16:17 state.cpt
-rw-------   1

## 6) Viewing a simulation with MDAnalysis and nglview

In [ ]:
import MDAnalysis as mda
import nglview

In [ ]:

ls -lrt

In [65]:
### Let's load the trp and trajectory file in the last simulation 

In [66]:
u = mda.Universe('peptidesim_nvt.tpr','traj.trr')
#print(u)
ag = u.select_atoms('protein')

view = nglview.show_mdanalysis(ag)
view.add_spacefill()

view


NGLWidget(max_frame=10)

In [67]:
view.color_by('atomindex')

In [68]:
from nglview.contrib.movie import MovieMaker
movie = MovieMaker(view, output='my.gif')
movie
